In [1]:
%cd /content/drive/MyDrive/Agriculture App/agriculture-predictor-planner

/content/drive/MyDrive/Agriculture App/agriculture-predictor-planner


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [39]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


# **Loading and Exploring the soil and weather dataset**

In [ ]:
file_path = 'data/raw/agri_app_base_dataset.xlsx'
data_weather_soil = pd.read_excel(file_path)

# Display the first few rows
print(data_weather_soil.info())
print(data_weather_soil.head(40))
print(data_weather_soil.shape)


In [ ]:
print(list(data_weather_soil.columns))


In [ ]:
print(data_weather_soil.dtypes)

In [ ]:
#Converting data types so that district code, year, and state code are not treated as numeric during calculations
data_weather_soil = data_weather_soil.astype({'Dist Code':'category', 'Year':'category', 'State Code':'category', 'State Name':'category', 'Dist Name':'category'})

print(data_weather_soil.dtypes)

In [ ]:
#dropping redundant columns with respect to crops dataset

columns_to_drop = ['RICE AREA (1000 ha)', 'RICE PRODUCTION (1000 tons)','RICE YIELD (Kg per ha)', 'PEARL MILLET AREA (1000 ha)', 'PEARL MILLET PRODUCTION (1000 tons)','PEARL MILLET YIELD (Kg per ha)', 'CHICKPEA AREA (1000 ha)', 'CHICKPEA PRODUCTION (1000 tons)','CHICKPEA YIELD (Kg per ha)', 'GROUNDNUT AREA (1000 ha)', 'GROUNDNUT PRODUCTION (1000 tons)','GROUNDNUT YIELD (Kg per ha)', 'SUGARCANE AREA (1000 ha)', 'SUGARCANE PRODUCTION (1000 tons)', 'SUGARCANE YIELD (Kg per ha)' ]
data_weather_soil.drop(columns=columns_to_drop, inplace=True)
print(data_weather_soil.columns)


In [ ]:
print(data_weather_soil.shape)

In [ ]:
data_weather_soil.head(10)

**Handling missing values**

In [ ]:
# Check for missing values

print("\nMissing Values:")
print(data_weather_soil.isnull().sum())


In [92]:
#dropping irrelevant column wrt to the project
data_weather_soil.drop(columns='TOTAL AGRICULTURAL LABOUR POPULATION (1000 Number)', inplace=True)

In [ ]:
# Find columns with missing values
missing_values = data_weather_soil.isnull().sum()

# Filter columns that have missing values
missing_columns = missing_values[missing_values > 0]
print("Columns with Missing Values:")
print(missing_columns)

In [ ]:
# Identify numerical and categorical columns
numerical_cols = data_weather_soil.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = data_weather_soil.select_dtypes(include=['object']).columns

# Separate columns with missing values into numerical and categorical
missing_numerical_cols = [col for col in numerical_cols if col in missing_columns.index]
missing_categorical_cols = [col for col in categorical_cols if col in missing_columns.index]

print("\nNumerical Columns with Missing Values:")
print(missing_numerical_cols)

print("\nCategorical Columns with Missing Values:")
print(missing_categorical_cols)

In [ ]:
# Fill missing numerical values with the mean
for col in missing_numerical_cols:
    data_weather_soil[col] = data_weather_soil[col].fillna(data_weather_soil[col].mean())
    print(f"Filled missing values in numerical column '{col}' with mean: {data_weather_soil[col].mean()}")


# Check for any remaining missing values
remaining_missing = data_weather_soil.isnull().sum().sum()
if remaining_missing == 0:
    print("\nAll missing values have been handled.")
else:
    print(f"\nThere are still {remaining_missing} missing values remaining in the dataset.")



**Handling outliers**

In [ ]:
print("Summary statistics of numerical columns:")
print(data_weather_soil.describe())

In [ ]:
# Visualize the outliers using boxplot for each numerical column
for col in data_weather_soil.select_dtypes(include=['float64', 'int64']).columns:
    plt.figure(figsize=(6, 4))
    sns.boxplot(x=data_weather_soil[col])
    plt.title(f'Boxplot for {col}')
    plt.show()

In [ ]:
#Checking the distribution of the features
for column in data_weather_soil.select_dtypes(include='number').columns:
    plt.figure(figsize=(8, 4))  # Set the size of the plot
    plt.hist(data_weather_soil[column], bins=30, edgecolor='k', alpha=0.7)
    plt.title(f'Distribution of {column}')
    plt.xlabel(column)
    plt.ylabel('Frequency')
    plt.grid(True)
    plt.show()

In [102]:
# Function to calculate the number of outliers in each column
def count_outliers_iqr(df):
    outlier_counts = {}
    for column in df.select_dtypes(include=['float64', 'int64']).columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
        outlier_counts[column] = len(outliers)
    return outlier_counts

# Get the count of outliers for each column
outlier_counts = count_outliers_iqr(data_weather_soil)
print("Number of outliers in each column using IQR method:")
print(outlier_counts)

Number of outliers in each column using IQR method:
{'GROSS CROPPED AREA (1000 ha)': 108, 'NITROGEN CONSUMPTION (tons)': 124, 'PHOSPHATE CONSUMPTION (tons)': 134, 'POTASH CONSUMPTION (tons)': 769, 'TOTAL FERTILISER CONSUMPTION (tons)': 125, 'GROSS IRRIGATED AREA (1000 ha)': 192, 'JANUARY MAXIMUM TEMPERATURE (Centigrate)': 0, 'FEBRUARY MAXIMUM TEMPERATURE (Centigrate)': 1, 'MARCH MAXIMUM TEMPERATURE (Centigrate)': 96, 'APRIL MAXIMUM TEMPERATURE (Centigrate)': 141, 'MAY MAXIMUM TEMPERATURE (Centigrate)': 59, 'JUNE MAXIMUM TEMPERATURE (Centigrate)': 12, 'JULY MAXIMUM TEMPERATURE (Centigrate)': 88, 'AUGUST MAXIMUM TEMPERATURE (Centigrate)': 83, 'SEPTEMBER MAXIMUM TEMPERATURE (Centigrate)': 200, 'OCTOBER MAXIMUM TEMPERATURE (Centigrate)': 97, 'NOVEMBER MAXIMUM TEMPERATURE (Centigrate)': 178, 'DECEMBER MAXIMUM TEMPERATURE (Centigrate)': 6, 'Winter JAN-FEB MAXIMUM TEMPERATURE (Centigrate)': 1, 'Summer MAR-MAY MAXIMUM TEMPERATURE (Centigrate)': 161, 'Rainy JUN-SEP MAXIMUM TEMPERATURE (Centigra

In [ ]:


# Function to replace outliers with median
def replace_outliers_with_median(df):
    # Iterate over numeric columns
    for column in df.select_dtypes(include=['number']).columns:
        # Calculate IQR
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        median = df[column].median()

        # Replace outliers with median
        df[column] = np.where((df[column] < lower_bound) | (df[column] > upper_bound), median, df[column])
    return df



# Replace outliers in all numeric columns
df = replace_outliers_with_median(data_weather_soil)
outlier_counts = count_outliers_iqr(df)
print("Number of outliers in each column using IQR method:")
print(outlier_counts)

print(df.head())


In [26]:
cleaned_path = 'data/cleaned/cleaned_soil_weather_data.xlsx'
data_weather_soil.to_excel(cleaned_path, index=False)
print(f"Cleaned soil and weather data saved to {cleaned_path}")


Cleaned soil and weather data saved to data/cleaned/cleaned_soil_weather_data.xlsx


# **Loading & Exploring multiple crops produce dataset**

In [ ]:
file_path = 'data/raw/district_crop_produce.csv'
data_crop_produce = pd.read_csv(file_path)

# Display the first few rows
print(data_crop_produce.info())
print(data_crop_produce.head(10))
print(data_crop_produce.shape)


In [ ]:
# Check for missing values

print("\nMissing Values:")
print(data_crop_produce.isnull().sum())


In [ ]:
print("Summary statistics of numerical columns:")
print(data_crop_produce.describe())

In [ ]:
# Visualize the outliers using boxplot for each numerical column
for col in data_crop_produce.select_dtypes(include=['float64', 'int64']).columns:
    plt.figure(figsize=(6, 4))
    sns.boxplot(x=data_crop_produce[col])
    plt.title(f'Boxplot for {col}')
    plt.show()

In [ ]:
# Apply IQR-based outlier removal for all numerical columns
for col in data_crop_produce.select_dtypes(include=['float64', 'int64']).columns:
    original_size = data_crop_produce.shape[0]
    data_crop_produce = remove_outliers_iqr(data_crop_produce, col)
    new_size = data_crop_produce.shape[0]
    print(f"Removed {original_size - new_size} outliers from {col}.")


In [32]:
cleaned_path = 'data/cleaned/cleaned_crop_produce_data.csv'
data_crop_produce.to_csv(cleaned_path, index=False)
print(f"Cleaned crop produce data saved to {cleaned_path}")


Cleaned crop produce data saved to data/cleaned/cleaned_crop_produce_data.csv
